In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import collections
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1854,tt0158622,83000000,59468275,The Flintstones in Viva Rock Vegas,Mark Addy|Stephen Baldwin|Kristen Johnston|Jan...,Brian Levant,Get ready to rock!,The Flintstones are at it again. The Flintston...,90,Science Fiction|Comedy|Family|Romance,Universal Pictures|Amblin Entertainment,4/28/2000,4.4,2000
859,tt1602098,8000000,5634828,Albert Nobbs,Glenn Close|Mia Wasikowska|Aaron Taylor-Johnso...,Rodrigo GarcÃ­a,A man with a secret. A woman with a dream.,Albert Nobbs struggles to survive in late 19th...,113,Drama,Chrysalis Films,12/21/2011,6.1,2011
234,tt2183034,13000000,45300000,Earth to Echo,Teo Halm|Astro|Resse Hartwig|Ella Wahlestedt|J...,Dave Green,No one will ever believe our story.,After a construction project begins digging in...,89,Family|Adventure|Science Fiction,Walt Disney Studios Motion Pictures|Panay Films,7/2/2014,5.7,2014
775,tt1253864,75000000,226904017,Immortals,Mickey Rourke|Kellan Lutz|Isabel Lucas|Henry C...,Tarsem Singh,The Gods need a hero.,Theseus is a mortal man chosen by Zeus to lead...,110,Fantasy|Action|Drama,Virgin Produced|Mark Canton Productions,11/10/2011,5.6,2011
1288,tt1821549,12000000,17654912,Nebraska,Bruce Dern|Will Forte|Bob Odenkirk|June Squibb...,Alexander Payne,Life's not about winning or losing. It's about...,"An aging, booze-addled father takes a trip fro...",115,Drama|Adventure,Paramount Vantage|Bona Fide Productions,9/21/2013,7.3,2013


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

# выделяем дату создания фильма
def count_common_data(data):
    count_data = collections.Counter()
    new_data = data.str.split('|', expand = False)
    for lists in new_data:
        for word in lists:
            count_data[word]+=1
    return count_data.most_common(1)

def count_director_movie(movie_bd, x):
    data_dir = movie_bd[x].str.cat(sep='|')
    clear_data = pd.Series(data_dir.split('|'))
    data = clear_data.value_counts(ascending=False)
    return data

# выносим profit
data['profit'] = data['revenue'] - data['budget']
# выделяем дату создания фильма
data['release_date'] = pd.to_datetime(data['release_date']) 
# выделяем месяц создания фильма
data['release_month'] = data['release_date'].dt.month
# выделяем количество символов в названии
data['title_length'] = data['original_title'].map(lambda x: len(x))
# выделяем количество символов в описании
data['length_overview'] = data.overview.map(lambda x: len(x.split(" ")))

# выделяем данные по компаниям
production_companies_data = pd.DataFrame(data['production_companies'].str.split('|').tolist()).stack().to_frame()
new_data = data.reset_index()
new_production_companies_data = production_companies_data.reset_index()
new_production_companies_data.columns = ['index', 'index_1', 'company']
data_join = new_data.merge(new_production_companies_data, on = 'index', how = 'left')

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

# 1. У какого фильма из списка самый большой бюджет?

In [11]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [6]:
data.original_title[data.budget.idxmax()]

'Pirates of the Caribbean: On Stranger Tides'

In [7]:
data[['budget','original_title','imdb_id']].sort_values('budget', ascending=False)[:10]

,budget,original_title,imdb_id
723,380000000,Pirates of the Caribbean: On Stranger Tides,tt1298650
1669,300000000,Pirates of the Caribbean: At World's End,tt0449088
14,280000000,Avengers: Age of Ultron,tt2395427
1014,260000000,John Carter,tt0401729
380,260000000,Tangled,tt0398286
1676,258000000,Spider-Man 3,tt0413300
1245,255000000,The Lone Ranger,tt1210819
374,250000000,Harry Potter and the Deathly Hallows: Part 1,tt0926084
1186,250000000,The Hobbit: The Desolation of Smaug,tt1170358
972,250000000,The Dark Knight Rises,tt1345836


# 2. Какой из фильмов самый длительный (в минутах)?

In [12]:
answers['2'] = 'Gods and Generals (tt0279111)'

In [8]:
data.original_title[data.runtime.idxmax()]

'Gods and Generals'

In [9]:
data[['runtime','original_title','imdb_id']].sort_values('runtime', ascending=False)[:10]

,runtime,original_title,imdb_id
1157,214,Gods and Generals,tt0279111
1081,201,The Lord of the Rings: The Return of the King,tt0167260
1736,191,Grindhouse,tt0462322
1336,187,King Kong,tt0360717
504,183,Pearl Harbor,tt0213149
1183,180,The Wolf of Wall Street,tt0993846
863,179,The Lord of the Rings: The Two Towers,tt0167261
496,178,The Lord of the Rings: The Fellowship of the Ring,tt0120737
1601,175,Alexander,tt0346491
993,172,Cloud Atlas,tt1371111


# 3. Какой из фильмов самый короткий (в минутах)?





In [13]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

In [14]:
data.original_title[data.runtime.idxmin()]

'Winnie the Pooh'

In [15]:
data[['runtime','original_title','imdb_id']].sort_values('runtime', ascending=True)[:10]

,runtime,original_title,imdb_id
768,63,Winnie the Pooh,tt1449283
930,72,Return to Never Land,tt0280030
1097,72,The Jungle Book 2,tt0283426
1408,76,Valiant,tt0361089
1626,76,Home on the Range,tt0299172
1348,77,Corpse Bride,tt0121164
1807,78,The Emperor's New Groove,tt0120917
884,78,Stuart Little 2,tt0243585
252,79,9,tt0472033
438,80,Jonah Hex,tt1075747


# 4. Какова средняя длительность фильмов?


In [16]:
answers['4'] = '109.658549'

In [17]:
data.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [18]:
answers['5'] = '107.0'

In [19]:
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [20]:
answers['6'] = 'Avatar (tt0499549)'

In [21]:
data.loc[data.profit==data.profit.max(),'original_title'].values[0]

'Avatar'

In [22]:
data[['profit','original_title','imdb_id']].sort_values('profit', ascending=False)[:10]

,profit,original_title,imdb_id
239,2544505847,Avatar,tt0499549
3,1868178225,Star Wars: The Force Awakens,tt2488496
0,1363528810,Jurassic World,tt0369610
4,1316249360,Furious 7,tt2820852
970,1299557910,The Avengers,tt0848228
722,1202817822,Harry Potter and the Deathly Hallows: Part 2,tt1201607
14,1125035767,Avengers: Age of Ultron,tt2395427
1177,1124219009,Frozen,tt2294629
8,1082730962,Minions,tt2293640
1081,1024888979,The Lord of the Rings: The Return of the King,tt0167260


# 7. Какой фильм самый убыточный? 

In [23]:
answers['7'] = 'The Lone Ranger (tt1210819)'

In [24]:
data.sort_values(by='profit').head(10)['original_title']

1245                 The Lone Ranger
1616                       The Alamo
809                  Mars Needs Moms
958     The Adventures of Pluto Nash
1464                    Flushed Away
636                        Australia
1872                       Supernova
1400              A Sound of Thunder
1718                           Awake
468                  How Do You Know
Name: original_title, dtype: object

In [25]:
data[['profit','original_title','imdb_id']].sort_values('profit', ascending=True)[:10]

,profit,original_title,imdb_id
1245,-165710090,The Lone Ranger,tt1210819
1616,-119180039,The Alamo,tt0318974
809,-111007242,Mars Needs Moms,tt1305591
958,-92896027,The Adventures of Pluto Nash,tt0180052
1464,-84540684,Flushed Away,tt0424095
636,-80445998,Australia,tt0455824
1872,-75171919,Supernova,tt0134983
1400,-74010360,A Sound of Thunder,tt0318081
1718,-71626175,Awake,tt0211933
468,-71331093,How Do You Know,tt1341188


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [26]:
answers['8'] = '1478'

In [27]:
data.loc[(data.profit)>0].shape[0]

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [28]:
answers['9'] = 'The Dark Knight (tt0468569)'

In [29]:
data.loc[data.release_year == 2008].set_index('original_title')['profit'].idxmax()

'The Dark Knight'

In [30]:
data[['revenue','original_title','imdb_id']][data['release_year']==2008].sort_values('revenue', ascending=False)[:10]

,revenue,original_title,imdb_id
599,1001921825,The Dark Knight,tt0468569
603,786636033,Indiana Jones and the Kingdom of the Crystal S...,tt0367882
606,631744560,Kung Fu Panda,tt0441773
621,624029371,Hancock,tt0448157
607,609841637,Mamma Mia!,tt0795421
612,603900354,Madagascar: Escape 2 Africa,tt0479952
605,586090727,Quantum of Solace,tt0830515
601,585174222,Iron Man,tt0371746
600,521311860,WALLÂ·E,tt0910970
624,419651413,The Chronicles of Narnia: Prince Caspian,tt0499448


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [31]:
answers['10'] = 'The Lone Ranger (tt1210819)'

In [32]:
data.loc[(data.release_year>=2012)&(data.release_year<=2014)].sort_values(by='profit')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,title_length,length_overview
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,2013-07-03,6.0,2013,-165710090,7,15,68
1214,tt0790736,130000000,61648500,R.I.P.D.,Jeff Bridges|Ryan Reynolds|Kevin Bacon|Stephan...,Robert Schwentke,To protect and serve the living,A recently slain cop joins a team of undead po...,96,Fantasy|Action|Comedy|Crime,Universal Pictures|Original Film|Dark Horse En...,2013-07-18,5.3,2013,-68351500,7,8,35
1007,tt1374992,60000000,8106475,Upside Down,Kirsten Dunst|Jim Sturgess|Timothy Spall|James...,Juan Diego Solanas,Two worlds. One future.,In an alternate universe where twinned worlds ...,104,Romance|Science Fiction|Drama|Fantasy,Onyx Films|Studio 37|Jouror Productions|France...,2012-09-27,6.1,2012,-51893525,9,11,50
1302,tt0884726,70000000,18662027,Legends of Oz: Dorothy's Return,Lea Michele|Dan Aykroyd|Patrick Stewart|Hugh D...,Dan St. Pierre,There's trouble in OZ...,"Dorothy wakes up in post-tornado Kansas, only ...",88,Animation|Music|Family,Prana Animation Studios|Summertime Entertainment,2013-06-13,6.1,2013,-51337973,6,31,68
1235,tt1308729,55000000,9489829,Bullet to the Head,Sylvester Stallone|Sung Kang|Sarah Shahi|Adewa...,Walter Hill,Revenge Never Gets Old.,"After watching their respective partners die, ...",92,Action|Crime|Thriller,Dark Castle Entertainment|After Dark Films|IM ...,2013-01-31,5.2,2013,-45510171,1,18,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,tt1690953,76000000,970761885,Despicable Me 2,Steve Carell|Kristen Wiig|Benjamin Bratt|Miran...,Pierre Coffin|Chris Renaud,Back 2 Work,Gru is recruited by the Anti-Villain League to...,98,Animation|Comedy|Family,Universal Pictures|Illumination Entertainment,2013-06-25,7.0,2013,894761885,6,15,16
974,tt1074638,200000000,1108561013,Skyfall,Daniel Craig|Judi Dench|Javier Bardem|Ralph Fi...,Sam Mendes,Think on your sins.,When Bond's latest assignment goes gravely wro...,143,Action|Adventure|Thriller,Columbia Pictures,2012-10-25,6.8,2012,908561013,10,7,94
1180,tt1300854,200000000,1215439994,Iron Man 3,Robert Downey Jr.|Gwyneth Paltrow|Guy Pearce|D...,Shane Black,Unleash the power behind the armor.,When Tony Stark's world is torn apart by a for...,130,Action|Adventure|Science Fiction,Marvel Studios,2013-04-18,6.9,2013,1015439994,4,10,22
1177,tt2294629,150000000,1274219009,Frozen,Kristen Bell|Idina Menzel|Jonathan Groff|Josh ...,Chris Buck|Jennifer Lee,Only the act of true love will thaw a frozen h...,Young princess Anna of Arendelle dreams about ...,102,Animation|Adventure|Family,Walt Disney Pictures|Walt Disney Animation Stu...,2013-11-27,7.5,2013,1124219009,11,6,66


In [33]:
data[data.profit==data.query('2012 <= release_year <= 2014').profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,title_length,length_overview
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,2013-07-03,6.0,2013,-165710090,7,15,68


# 11. Какого жанра фильмов больше всего?

In [34]:
answers['11'] = 'Drama'

In [35]:
count_common_data(data['genres'])

[('Drama', 782)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [36]:
answers['12'] = 'Drama'

In [37]:
count_common_data(data[data['profit'] > 0]['genres'])

[('Drama', 560)]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [38]:
answers['13'] = 'Peter Jackson'

In [39]:
data.groupby('director')[['revenue']].sum().sort_values('revenue', ascending = False).head(1)

,revenue
director,
Peter Jackson,6490593685


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [40]:
answers['14'] = 'Robert Rodriguez'

In [41]:
data_dir_action = count_director_movie(data[data['genres'].map(lambda x: True if 'Action' in x else False)],'director')
print(data_dir_action)

Robert Rodriguez         9
Michael Bay              7
Paul W.S. Anderson       7
Ridley Scott             6
Antoine Fuqua            6
                        ..
Gina Prince-Bythewood    1
James Mather             1
Michael Spierig          1
Phil Joanou              1
Rob Minkoff              1
Length: 364, dtype: int64


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [42]:
answers['15'] = 'Chris Hemsworth'

In [43]:
release_year_data = data[data['release_year'] == 2012]

revenue_count = Counter()
for i in range(0,len(release_year_data)):
    for p in release_year_data.iloc[i]['cast'].split('|'):
        revenue_count[p] += release_year_data.iloc[i]['revenue']
revenue_count.most_common(1)

[('Chris Hemsworth', 2027450773)]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [44]:
answers['16'] = 'Matt Damon'

In [45]:
pd.Series(data[data.budget > data.budget.mean()].cast.str.split('|').sum()).value_counts().index[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [46]:
answers['17'] = 'Action'

In [47]:
Counter(data[data.cast.str.contains('Nicolas Cage')].genres.str.split('|').sum()).most_common(1)

[('Action', 17)]

# 18. Самый убыточный фильм от Paramount Pictures

In [48]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

In [49]:
data[data.production_companies.str.contains('Paramount Pictures')][['original_title', 'profit','imdb_id']].sort_values('profit', ascending = True).head(1)

,original_title,profit,imdb_id
925,K-19: The Widowmaker,-64831034,tt0267626


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [50]:
answers['19'] = '2015'

In [51]:
pd.DataFrame(data.groupby(by = 'release_year').revenue.sum()).sort_values('revenue', ascending = False)

,revenue
release_year,
2015,25449202382
2014,23405862953
2013,23213799791
2012,23079001687
2011,22676791872
2010,21071204961
2009,20261791024
2008,18252781990
2007,18162406801


# 20. Какой самый прибыльный год для студии Warner Bros?

In [52]:
answers['20'] = '2014'

In [53]:
warner_bros_data = data[data.production_companies.str.contains('Warner Bros')]
pd.DataFrame(warner_bros_data.groupby(by = 'release_year').profit.sum()).sort_values('profit', ascending = False)

,profit
release_year,
2014,2295464519
2007,2201675217
2008,2134595031
2010,1974712985
2011,1871393682
2003,1855493377
2009,1822454136
2013,1636453400
2004,1631933725


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [54]:
answers['21'] = 'Сентябрь'

In [55]:
data[['imdb_id','release_month']].groupby(by = 'release_month').count().sort_values('imdb_id', ascending = False)

,imdb_id
release_month,
9,227
12,190
10,186
8,161
3,156
4,149
6,147
11,146
7,142


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [56]:
answers['22'] = '450'

In [71]:
len(data[data.release_month.isin([6, 7, 8])])

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [58]:
answers['23'] = 'Peter Jackson'

In [59]:
data.query('release_month in ["12", "1", "2"]')[['director','imdb_id']].groupby(by = 'director').count().sort_values('imdb_id', ascending = False)

,imdb_id
director,
Peter Jackson,7
Steven Soderbergh,6
Clint Eastwood,6
Adam Shankman,4
Martin Scorsese,4
...,...
GÃ¡bor CsupÃ³,1
Guy Ritchie,1
Guy Ferland,1


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [60]:
answers['24'] = 'Four By Two Productions'

In [61]:
data_join[['company', 'title_length']].groupby('company').mean().sort_values('title_length', ascending = False).head(1)

,title_length
company,
Four By Two Productions,83.0


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [62]:
answers['25'] = 'Midnight Picture Show'

In [63]:
data_join[['company', 'length_overview']].groupby('company').mean().sort_values('length_overview', ascending = False).head(1)

,length_overview
company,
Midnight Picture Show,175.0


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [64]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

In [65]:
data.loc[data['vote_average']>data.quantile(0.99, numeric_only=True)['vote_average']]['original_title']

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [66]:
answers['27'] = 'Daniel Radcliffe Rupert Grint'

In [67]:
doubles = data.cast.apply(lambda x: list(tuple(sorted(dob)) for dob in combinations(x.split('|'), 2))).tolist()
frequency = pd.DataFrame(doubles).stack()
frequency.value_counts()

(Daniel Radcliffe, Emma Watson)        8
(Daniel Radcliffe, Rupert Grint)       8
(Emma Watson, Rupert Grint)            8
(Ben Stiller, Owen Wilson)             6
(Helena Bonham Carter, Johnny Depp)    6
                                      ..
(Amber Heard, Nicolas Cage)            1
(Ricky Gervais, Tim Curry)             1
(Aunjanue Ellis, Julianne Moore)       1
(Rachel Weisz, Robbie Coltrane)        1
(Angelina Jolie, Geoff Pierson)        1
Length: 17943, dtype: int64

# Submission

In [68]:
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '109.658549',
 '5': '107.0',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe Rupert Grint'}

In [69]:
len(answers)

27